# Frequency-switched Data Reduction
-----------------------------------

This notebook shows how to use `dysh` to calibrate a frequency switched observations. The idea is similar to an OnOff observation, except the telescope does not move to an Off in position, but move their IF in frequency space. Here we call the On/Off the Sig and Ref. Since the telescope observes in a Track mode, combining the sig and a shifted (folded) ref, a $\sqrt{2}$ improvement in S/N can be achieved.

It retrieves and calibrates frequency-switched scans using `GBTFITSLoad.getfs()`, which returns a `ScanBlock` object.  

In [ ]:
import os
import wget
import astropy.units as u
from dysh.fits.gbtfitsload import GBTFITSLoad

## First, we download the example SDFITS data, if necessary.


In [ ]:
# just scan6 
filename = "AGBT20B_014_03.raw.vegas.A6.fits"
if not os.path.isfile(filename):
    url = f"http://www.gb.nrao.edu/dysh/example_data/onoff-L/data/{filename}"
    print(f"Downloading {filename}")
    wget.download(url,out=filename)
    print(f"\nRetrieved {filename}")
else:
    print(f"{filename} already downloaded")

In [ ]:
sdfits = GBTFITSLoad(filename)
sdfits.info()
sdfits.summary()

In [ ]:
#fsscan = sdfits.getfs(6, ifnum=0, plnum=1)
#    why does this not work, where getps(6) would work???
fsscan = sdfits.getfs(scan=6, ifnum=0, plnum=1)
fsscan.calibrate()

In [ ]:

print(f"T_sys = {np.nanmean(fsscan[0].tsys):.2f}")

In [ ]:
ta = fsscan.timeaverage(weights='tsys')

In [ ]:
ta.plot()

In [ ]:
ta.plot(xaxis_unit="km/s",yaxis_unit="K",ymin=-10,ymax=20,xmin=-2000,xmax=2000)

In [ ]:
kms = u.km/u.s
#ta.baseline(degree=2,exclude=[(-2000*kms,-400*kms),(-200*kms,100*kms),(600*kms,3000*kms)],remove=True)
ta.baseline(degree=2,exclude=[-200*kms,100*kms],remove=True)

# @todo  we really should have an include= option as well, mutually exclusive

In [ ]:
ta.plot(ymin=-1,ymax=1)

In [ ]:
print(ta.baseline_model)

In [ ]:
ta.savefig("baselined_removed.png")